# 1

In [75]:
function s8a_merge(v, idx_1, idx_2)
    vout = Array{Int64, 1}()
    cnt = 0
    
    while length(idx_1) != 0 && length(idx_2) != 0
        if v[idx_1[1]] > v[idx_2[1]]
            cnt += length(idx_1)
            push!(vout, popfirst!(idx_2))
        else
            push!(vout, popfirst!(idx_1))
        end
    end
    vout = vcat(vout, idx_1)    
    vout = vcat(vout, idx_2)

    return cnt, vout
end

function s8a(v, p, q)
    if (q-p) == 1
        if v[p] < v[q]
            return 0, [p;q]
        else
            return 1, [q;p]
        end
    end
    
    r = convert(Int64, floor((q-p)/2))
    
    cnt_1, idx_1 = s8a(v, p, p+r)
    cnt_2, idx_2 = s8a(v, p+r+1, q)
    cnt_3, idx = s8a_merge(v, idx_1, idx_2) 
    
    # idx no es necesario.
    return (cnt_1+cnt_2+cnt_3), idx
end

function s8a_wrapper(v)
    return s8a(v, 1, length(v))
end

s8a_wrapper (generic function with 1 method)

# 1
#### Versión O(n^2) en el peor caso (lista ordenada), pero q en listas muy desordenadas debería ir más rápido q la versión anterior.

In [41]:
function s8b_merge(v, idx_1, idx_2)
    vout = Array{Int64, 1}()
    cnt = 0
    
    for i in 1:length(idx_1)
        for j in length(idx_2):-1:1
            if v[idx_1[i]] > v[idx_2[j]]
                cnt += (length(idx_1) - i + 1) * j
                vout = vcat(vout, idx_2[1:j])
                deleteat!(idx_2, 1:j)
                break
            end
        end
        push!(vout, idx_1[i])     
    end
    vout = vcat(vout, idx_2)
    
    return cnt, vout
end

function s8b(v, p, q)
    if (q-p) == 1
        if v[p] < v[q]
            return 0, [p;q]
        else
            return 1, [q;p]
        end
    end
    
    r = convert(Int64, floor((q-p)/2))
    
    cnt_1, idx_1 = s8b(v, p, p+r)
    cnt_2, idx_2 = s8b(v, p+r+1, q)
    cnt_3, idx = s8b_merge(v, idx_1, idx_2) 
    
    # idx no es necesario.
    return (cnt_1+cnt_2+cnt_3), idx
end

function s8b_wrapper(v)
    return s8b(v, 1, length(v))
end

s8b_wrapper (generic function with 1 method)

In [80]:
v = rand(collect(1:100), 16);

In [81]:
s_a, id_a = s8a_wrapper(v)

(44, [4, 2, 15, 5, 7, 6, 3, 9, 16, 13, 8, 14, 1, 11, 10, 12])

In [82]:
s_b, id_b = s8b_wrapper(v)

(44, [4, 2, 15, 5, 7, 6, 3, 9, 16, 13, 8, 14, 1, 11, 10, 12])

#### uso `id` p/ confirmar que el conteo esté bien

In [83]:
# Guarda q el sortperm() de Julia no es estable. Si da F, generar un nuevo `v`
id_a == sortperm(v)

true

In [84]:
# Guarda q el sortperm() de Julia no es estable. Si da F, generar un nuevo `v`
id_b == sortperm(v)

true

# 2
#### Guarda que está en python. El resultado es una lista con todos los costos, el mínimo de esa lista será el óptimo.

In [66]:
def s3b(G, v, actual, n, count, cost, costos): 
    if (count == n and G[actual][0]): 
        costos.append(cost + G[actual][0]) 
        return

    for i in range(n):
        if (visitada[i] == False and G[actual][i]): 
            
            visitada[i] = True
            s3b(G, visitada, i, n, count + 1, 
                cost + G[actual][i], costos) 
            
            # backtrack
            visitada[i] = False

84

In [67]:
n = 4
G = [[ 0, 10, 15, 20 ], 
        [ 10, 0, 35, 25 ], 
        [ 15, 35, 0, 30 ], 
        [ 20, 25, 30, 0 ]]
G = [ [ 0, 5, 10, 15 ], 
     [ 5, 0, 30, 20 ],
     [ 10, 30, 0, 25 ],
     [ 15, 20, 25, 0 ] ]

visitada = [False for i in range(n)] 
visitada[0] = True
costos = []

14-element Array{Int64,1}:
 36
 55
 68
 17
 33
 59
  8
 61
 35
 81
 99
 19
 85
 97

In [ ]:
s3b(G, visitada, 0, n, 1, 0, costos)

In [ ]:
costos

# 3
#### En su momento no hice la top-down. La pueden hacer uds.

In [85]:
function s6_rec(v, M)
    t = typemax(Int64)
    
    if v < 1
        return 0
    else
        for m in M
            if m <= v
                c = s6_rec(v-m, M)
                t = min(t, c+1)        
            end
        end
        return t
    end
end

s6_rec (generic function with 1 method)

In [89]:
function s6_bottom_up(v, todos_los_billetes)
    L = fill(typemax(Int64), v+1)
    L[1] = 0
    for subproblema in 2:v+1
#         println("vuelto: ", subproblema - 1) # corrijo 1-indexing
        for billete in todos_los_billetes
            resto = subproblema - billete # corrijo 1-indexing. El subproblema 2, es devolver vuelto = 1
            if resto > 0 # corrijo 1-indexing. El subproblema 2, es devolver vuelto = 1
                L[subproblema] = min(L[subproblema], 1 + L[resto])
                
#                 println("billetes: ", L[subproblema], " - L[resto] + 1:  ", L[resto], " - resto: ", resto - 1)
                
            end
        end
#         println("-----")
    end
    
    # L es el array de memo
    return hcat(collect(0:v), L)
    
    # El algoritmo sólo debería devolver el resultado del último subproblema
#     return L[end]
end

s6_bottom_up (generic function with 1 method)

In [91]:
M = [ 50 ; 20 ; 10 ; 8; 5 ; 2 ; 1]

7-element Array{Int64,1}:
 50
 20
 10
  8
  5
  2
  1

In [92]:
s6_rec(16, M)

2

#### El resultado también es 2, pero p/ q vean toda la lista llena,la imprimo (derecha) junto con los valores de los subproblemas (es decir, el vuelto q hay q dar, a la izquierda).

In [93]:
rtdo = s6_bottom_up(16, M)

17×2 Array{Int64,2}:
  0  0
  1  1
  2  1
  3  2
  4  2
  5  1
  6  2
  7  2
  8  1
  9  2
 10  1
 11  2
 12  2
 13  2
 14  3
 15  2
 16  2